In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")

In [5]:
from monary import Monary
import numpy as np
import pandas as pd
import bson
from monary.monary import mvoid_to_bson_id
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import cluster
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_extraction import DictVectorizer

In [6]:
mon = Monary("127.0.0.1")

# PA dataframe

In [7]:
columns = ["User_"+str(i) for i in range(31)]
array = mon.query(
    "newbase1",
    "private_area",
    {"_id":mvoid_to_bson_id('574840a9dfb7b83c907b35de')},
    ["members.%d.person._ref.$id" % i for i in range(31)],
    ["id" for i in range(31)]
    )
df_pa = np.matrix(array).transpose() 
#df_pa = pd.DataFrame(df_pa, columns=columns)

In [8]:
list_users_pse = []
for i in range(31):
    list_users_pse.append(mvoid_to_bson_id(df_pa[0,i]))

In [7]:
len(list_users_pse)

31

# Atlas List

In [9]:
arrays = mon.query(
"newbase1", # database name
"atlas", # collection name
{
        "name":"Probability and Statistics for Economic Analysis"
    }, # query spec
["content.nodes.%d.object._ref.$id"%i for i in range(5)], # field names (in Mongo record)
[
        "id" for i in range(5)] # Monary field types (see below)
)
df_atlas = np.matrix(arrays).transpose() 

In [10]:
list_atlas_prob_stat = []
for i in range(5):
    list_atlas_prob_stat.append(mvoid_to_bson_id(df_atlas[0,i]))

In [11]:
list_atlas_prob_stat

[ObjectId('57484755dfb7b83c907b77c5'),
 ObjectId('5748475fdfb7b83c907b797b'),
 ObjectId('57485589dfb7b83ecce1c292'),
 ObjectId('57f3dfbcd31463000b21a723'),
 ObjectId('57f3e1767357a5000b6afa02')]

# Map List

In [12]:
arrays = mon.query(
"newbase1", # database name
"map", # collection name
{
        "_id":{"$in":list_atlas_prob_stat}
    }, # query spec
["name"]+["graph.nodes.%d.object"%i for i in range(13)], # field names (in Mongo record)
["string:100"]+[
        "id" for i in range(13)] # Monary field types (see below)
)
df_map = np.matrix(arrays).transpose() 

In [13]:
from bson import ObjectId

In [120]:
mvoid_to_bson_id(df_map[2,1])

ObjectId('579632509288d4000b36ae66')

In [15]:
dictionary = {}
for i in range(len(list_atlas_prob_stat)):
    dictionary[df_map[i,0]]=[mvoid_to_bson_id(a)   for a in df_map.A[i,1:] if mvoid_to_bson_id(a) != mvoid_to_bson_id('000000000000000000000000')]

In [19]:
dictionary
shape = [len(a) for a in dictionary.values()]

In [17]:
list_caps_prob_stat = sum(dictionary.values())

In [20]:
df_map_name = {}
map_name = []
for number,name in enumerate(dictionary.keys()):
    map_name += [name]*shape[number]
df_map_name["map name"] = map_name
df_map_name["Capsule id"] = sum(dictionary.values())

In [21]:
df_map_name = pd.DataFrame.from_dict(df_map_name)

In [22]:
df_map_name.to_csv("BaseMapPSE")

# Capsule dataframe

In [74]:
columns = [
    "Capsule id",
    #"date created",
    #"date updated",
    "capsule name",
    "type of author",
    "author user id",
    "is draft",
    "is public",
    "approximate capsule duration",
]
arrays = mon.query(
"newbase1", # database name
"capsule", # collection name
{"publication.private_areas_sharing_list":{"$all":[mvoid_to_bson_id('574840a9dfb7b83c907b35de')]}}, # query spec
[
        "_id",
        #"date_created",
        #"date_updated",
        "name",
        "author._ref.$ref",
        "author._ref.$id",
        "publication.is_draft",
        "publication.is_public",
        "approximate_duration",

], # field names (in Mongo record)
[
        "id",
    #   "date",
        #"date",
        "string:100",
        "string:20",
        "id",
        "bool",
        "bool",
        "float32",
  
    ] # Monary field types (see below)
)
df_capsule = np.matrix(arrays).transpose() 

df_capsule = pd.DataFrame(df_capsule, columns=columns)

In [75]:
df_capsule["Capsule id"] = df_capsule["Capsule id"].apply(mvoid_to_bson_id)
df_capsule["author user id"]=df_capsule["author user id"].apply(mvoid_to_bson_id)
#df["date updated"]=df["date updated"].astype('int').astype("datetime64[ms]")
#df["date created"]=df["date created"].astype('int').astype("datetime64[ms]")
df_capsule["is draft"]=df_capsule["is draft"].apply(lambda x: 1 if x==True else 0)
df_capsule["is public"]=df_capsule["is public"].apply(lambda x: 1 if x==True else 0)
df_capsule["approximate capsule duration"]=df_capsule["approximate capsule duration"].astype('float')


# Export de la base de capsule

In [76]:
df_capsule.to_csv("BaseCapsulePSE")

# Base Utilisateur

In [27]:
columns_user = ["user id", "first name", "last name"]# , "date joined"]
arrays_user = mon.query(
"newbase1",
"user",
{"_id":{"$in":list_users_pse}},
["_id","first_name","last_name"], #,"date_joined"],
["id","string:30","string:30"]#,"date"],

)
df_user = np.matrix(arrays_user).transpose()
df_user = pd.DataFrame(df_user, columns=columns_user)
df_user["user id"] = df_user["user id"].apply(mvoid_to_bson_id)
df_user["user name"] = df_user["first name"] + " " + df_user["last name"]
#df_user["date joined"] = df_user["date joined"].astype("int")
df_user = df_user[["user id","user name"]]#, "date joined"]]
df_user

,user id,user name
0,564f68e80a728f08a779fe6e,Son Thierry Ly
1,564fa20a0a728f0a68b6b089,Arnaud Riegert
2,57485ef0dfb7b83ecce1f01c,Marc Gurgand
3,5762d0afdfb7b870f9ae71de,Vincent Runge
4,5762d0afdfb7b870f9ae71e1,Mohammed Sedki
5,57d68a8cda26e0000b00431c,Christian Abele
6,57d68a8dda26e0000b00431f,Catalina AGNES
7,57d68a8dda26e0000b004322,Léa Ambroise
8,57d68a8dda26e0000b004325,Awonon Josué AWONON
9,57d68a8dda26e0000b004328,Srinivas Balasubramanian


# Export de la base utilisateur 

In [28]:
df_user.to_csv("BaseUtilisateursPSE")

# Base de Test Attempts

In [51]:
columns_test_attempt = [
    "Test Attempt id",
    "date created",
    "date finished",
    "Capsule id",
    "user id",
    "is completed",
    "is succeeded",
    "score"

]
arrays_test_attempt = mon.query(
"newbase1", # database name
"test_attempt", # collection name
{"test_container._ref.$id":{"$in":list_caps_prob_stat}}, # query spec
[
        "_id",
        "date_created",
        "date_finished",
        "test_container._ref.$id",
        "user",
        "is_completed",
        "is_succeeded",
        "score"

], # field names (in Mongo record)
[
        "id",
        "date",
        "date",
        "id",
        "id",
        "bool",
        "bool",
        "float32"
  
    ] # Monary field types (see below)
)
df_test_attempt = np.matrix(arrays_test_attempt).transpose() 

df_test_attempt = pd.DataFrame(df_test_attempt, columns=columns_test_attempt)
df_test_attempt["Test Attempt id"] = df_test_attempt["Test Attempt id"].apply(mvoid_to_bson_id)
df_test_attempt["user id"]=df_test_attempt["user id"].apply(mvoid_to_bson_id)
df_test_attempt["Capsule id"]=df_test_attempt["Capsule id"].apply(mvoid_to_bson_id)
df_test_attempt["date finished"]=df_test_attempt["date finished"].astype('int').astype("datetime64[ms]")
df_test_attempt["date created"]=df_test_attempt["date created"].astype('int').astype("datetime64[ms]")
len(df_test_attempt["Capsule id"].value_counts())

30

In [52]:
df_test_attempt = df_test_attempt[df_test_attempt["user id"] != mvoid_to_bson_id('000000000000000000000000')]

# Sauvegarde de la base de Test Attempts

In [53]:
df_test_attempt.to_csv("BaseTestAttemptsPSE")

In [77]:
df_capsule_name = df_capsule[['Capsule id','capsule name']]

In [78]:
df_test_attempt_wth_caps_name = pd.merge(left=df_capsule_name,right=df_test_attempt,on="Capsule id",how ='inner')
df_test_attempt_wth_caps_name

,Capsule id,capsule name,Test Attempt id,date created,date finished,user id,is completed,is succeeded,score
0,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57c405f0854c7d000b48b57e,2016-08-29 09:52:48.703,2016-08-29 09:54:29.834,5762d0afdfb7b870f9ae71de,True,True,1
1,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57c52dcac10652000b1bbc80,2016-08-30 06:55:06.087,2016-08-30 06:55:26.366,5762d0afdfb7b870f9ae71de,True,False,0.6
2,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57c55294c10652000b2031ab,2016-08-30 09:32:04.362,2016-08-30 09:34:45.333,5762d0afdfb7b870f9ae71de,True,False,0.8
3,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57c6bc7dc4899d000b4c293f,2016-08-31 11:16:13.904,2016-08-31 11:21:43.050,5762d0afdfb7b870f9ae71de,True,True,1
4,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57c6cd05c4899d000b4d13a1,2016-08-31 12:26:45.931,2016-08-31 12:33:42.249,5762d0afdfb7b870f9ae71de,True,False,0.8
5,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57d070983a3c6a000b82fbaa,2016-09-07 19:55:04.410,2016-09-07 19:58:49.989,5762d0afdfb7b870f9ae71de,True,True,1
6,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57da6602c6d86d000bb03ab3,2016-09-15 09:12:34.129,2016-09-15 09:34:57.428,57d68a8dda26e0000b004349,True,False,0.4
7,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57da6b45c6d86d000bb13e3b,2016-09-15 09:35:01.925,2016-09-15 09:42:32.208,57d68a8dda26e0000b004349,True,True,0.8
8,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57df99769903a9000b5101eb,2016-09-19 07:53:26.780,2016-09-19 08:06:37.615,57d68a8dda26e0000b004331,True,False,0.7
9,57485297dfb7b83ecce16494,Hypothesis testing - Basics,57df9c9e9903a9000b5113c1,2016-09-19 08:06:54.288,2016-09-19 08:09:20.835,57d68a8dda26e0000b004331,True,True,0.9


In [79]:
df_ta_caps_user = pd.merge(left=df_test_attempt_wth_caps_name,right=df_user,on="user id",how ='inner')
df_ta_caps_user = df_ta_caps_user.sort_values(["Capsule id","user id","date created"])

In [80]:
list_users = dict(df_ta_caps_user["user name"].value_counts()).keys()
list_capsule = dict(df_ta_caps_user["capsule name"].value_counts()).keys()

In [81]:
df_ta_caps_user.to_csv('BaseTravailPSE')

In [38]:
def get_question_number(question):
    l = question.strip(']').split('[')
    n = len(l)
    return l[-1]

In [39]:
columns_questions = [
    "Capsule id",
    "date created",
    "date updated",
    "user id",
    "test attempt",
    "question number",
    "duration",
    "success"

]
arrays_questions = mon.query(
"newbase1", # database name
"statement", # collection name
{"verb.identifier": "http://adlnet.gov/expapi/verbs/answered",
"context.contextActivities.parent.object._ref.$id":{"$in":list_caps_prob_stat}}, # query spec
[
        "context.contextActivities.parent.0.object._ref.$id",
        "date_created",
        "date_updated",
        "actor.account.user",
        "object.definition.object._ref.$id",
        "object.definition.sub_element",
        "result.duration",
        "result.success"

], # field names (in Mongo record)
[
        "id",
        "date",
        "date",
        "id",
        "id",
        "string:100",
        "float32",
        "bool"
  
    ] # Monary field types (see below)
)
df_questions = np.matrix(arrays_questions).transpose() 

df_questions = pd.DataFrame(df_questions, columns=columns_questions)

In [40]:
df_questions["Capsule id"] = df_questions["Capsule id"].apply(mvoid_to_bson_id).astype("string")
df_questions["user id"]=df_questions["user id"].apply(mvoid_to_bson_id).astype("string")
df_questions["test attempt"]=df_questions["test attempt"].apply(mvoid_to_bson_id).astype("string")
df_questions["date updated"]=df_questions["date updated"].astype('int')
df_questions["date created"]=df_questions["date created"].astype('int')
df_questions["question number"]=df_questions["question number"].apply(get_question_number)
df_questions["success"]=df_questions["success"].apply(lambda x: 1 if x==True else 0)
df_questions["duration"]=df_questions["duration"].astype('float')

In [41]:
df_questions.to_csv("BaseQuestionsPSE")

In [45]:
len(df_questions["Capsule id"].value_counts())

30

In [47]:
df = df_questions[df_questions.columns-["test attempt","user id"]]

/home/enguerand/.virtualenvs/gryzz-lab/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  if __name__ == '__main__':


In [48]:
df.head()

,Capsule id,date created,date updated,duration,question number,success
0,579632509288d4000b36ae66,1471860312922,1471860312932,5.127562,0,0
1,579632509288d4000b36ae66,1471860315971,1471860315991,2.364411,1,0
2,579632509288d4000b36ae66,1471962537644,1471962537656,17.119530,0,0
3,579632509288d4000b36ae66,1471962550835,1471962550851,11.182281,1,0
4,579632509288d4000b36ae66,1472020770719,1472020770731,55.220470,0,0


## Import des bases de données


In [50]:
df_capsule = pd.read_csv("BaseCapsule", index_col=0)
df_user = pd.read_csv("BaseUtilisateurs",index_col=0)
df_test_attempt = pd.read_csv("BaseTestAttempts",index_col=0)
df_questions = pd.read_csv("BaseQuestions",index_col=0)


In [51]:
df_test_attempt = pd.read_csv("BaseTestAttempts",index_col=0)
df_test_attempt = df_test_attempt.rename(columns={'Test Attempt id':'test attempt'})[['test attempt','is completed','is succeeded','score']]
df_test_attempt

,test attempt,is completed,is succeeded,score
0,57288acd0a728f7346c5567c,True,True,1.000000
1,564dc8680a728f755ac642b9,True,False,1.000000
2,564dc8e60a728f755ac64334,False,False,0.000000
3,564dc8e60a728f755ac64341,True,False,1.000000
4,564dc90d0a728f7559e66b53,False,False,0.000000
5,564dc90d0a728f755ac643ad,True,False,1.000000
6,564dca210a728f7559e66c60,True,False,1.000000
7,564f33430a728f02c3d0278b,False,False,0.000000
8,564f33430a728f02c4f1be82,False,False,0.000000
9,57286b350a728f6f1756889c,False,False,0.000000


In [52]:
df_questions_user = pd.merge(left=df_user,right=df_questions,on="user id",how ='inner')

In [53]:
df_questions_user_capsule = pd.merge(left=df_capsule,right=df_questions_user,on="Capsule id",how ='inner')

In [54]:
df_user = df_user.rename(columns={'user id': 'author user id','user name':'author user name'})

In [55]:
df_questions_user_author_capsule = pd.merge(left=df_user[["author user id","author user name"]],right=df_questions_user_capsule,on="author user id",how ='inner')

In [56]:
df = pd.merge(left=df_test_attempt,right=df_questions_user_author_capsule,on="test attempt",how='inner')
df["is completed"]=df["is completed"].apply(lambda x: 1 if x==True else 0)
df["is succeeded"]=df["is succeeded"].apply(lambda x: 1 if x==True else 0)

In [57]:
df["is user author"] = (df["author user id"] == df["user id"]).apply(lambda x: 1 if x==True else 0)

In [58]:
df.to_csv("BaseFinale")

# Commencer ici

In [59]:
df = pd.read_csv("BaseFinale",index_col=0)

# Base de travail

In [60]:
new_columns = [
    "is completed",
    "is succeeded",
    "score",
    "type of author",
    "is draft",
    "is public",
    'approximate capsule duration',
    "date joined",
    "date created",
    "date updated",
    "question number",
    "duration",
    "success",
    "is user author"
]
df = df[new_columns]
df.head()

,is completed,is succeeded,score,type of author,is draft,is public,approximate capsule duration,date joined,date created,date updated,question number,duration,success,is user author
0,1,1,1.0,user,1,0,0.0,1454539430147,1467817861110,1467817861130,0,30.591999,1,1
1,1,1,1.0,user,1,0,0.0,1454539430147,1467817861140,1467817861143,1,23.615999,1,1
2,1,1,1.0,user,1,0,0.0,1454539430147,1467818354705,1467818354724,0,30.591999,1,1
3,1,1,1.0,user,1,0,0.0,1454539430147,1467818354734,1467818354737,1,23.615999,1,1
4,1,0,1.0,user,0,0,600.0,1448059402000,1467817860588,1467817860633,0,5.590000,1,0


# Traitement Base de Données

In [46]:
df_infos = pd.read_csv("BaseQuestionsPSE",index_col=0)
df_map_name = pd.read_csv("BaseMapPSE",index_col=0)
df_capsule = pd.read_csv("BaseCapsulePSE",index_col=0)

In [47]:
df_map_name

,Capsule id,map name
0,57973251630267004d21e2cf,2. Point estimation
1,57973280630267004d21ee5c,2. Point estimation
2,57973238630267004d21dd0a,2. Point estimation
3,57973529630267004d223659,2. Point estimation
4,579734f6630267004d222ac2,2. Point estimation
5,57973621630267004d224e03,2. Point estimation
6,57c75e05baefb4000be06d01,2. Point estimation
7,57973251630267004d21e2cf,2bis. Point estimation (Complete version)
8,57973280630267004d21ee5c,2bis. Point estimation (Complete version)
9,57973238630267004d21dd0a,2bis. Point estimation (Complete version)


In [50]:
df_questions = pd.merge(left=df_infos,right=df_map_name,on="Capsule id")
df_questions.to_csv("BaseQuestionsNomsPSE")
properties = df_questions[df_questions["map name"] == "1.1. Properties of random samples"]
properties = pd.merge(left=properties,right=df_capsule,on="Capsule id")

In [49]:
df_questions[df_questions['map name'] == '1. Properties of random samples']["Capsule id"].value_counts()

579633bc9288d4000b36f648    744
579632509288d4000b36ae66    709
579633a39288d4000b36f096    649
57964209630267000bbea3fa    649
5796432e630267000bbf30ad    639
579633209288d4000b36df6e    578
5796418e630267000bbe820d    577
579633069288d4000b36d9c0    574
57b5b38651f2db000b2f2527    540
5796428a630267000bbf07bc    533
579642a8630267000bbf19b2    518
5797399f630267004d22b8a1    486
579633819288d4000b36eae5    431
Name: Capsule id, dtype: int64

In [79]:
mvoid_to_bson_id(df_map[2,3])

ObjectId('080000000000000090f7d5eb')

In [12]:
pd.read_csv("BaseQuestionsNomsPSE",index_col=0)

,Capsule id,date created,date updated,user id,test attempt,question number,duration,success,map name
0,579632509288d4000b36ae66,1471860312922,1471860312932,564f68e80a728f08a779fe6e,57bace537fc49a000bd84221,0,5.127562,0,1.1. Properties of random samples
1,579632509288d4000b36ae66,1471860315971,1471860315991,564f68e80a728f08a779fe6e,57bace537fc49a000bd84221,1,2.364411,0,1.1. Properties of random samples
2,579632509288d4000b36ae66,1471962537644,1471962537656,564f68e80a728f08a779fe6e,57bc5d981a68b1000b171473,0,17.119530,0,1.1. Properties of random samples
3,579632509288d4000b36ae66,1471962550835,1471962550851,564f68e80a728f08a779fe6e,57bc5d981a68b1000b171473,1,11.182281,0,1.1. Properties of random samples
4,579632509288d4000b36ae66,1472020770719,1472020770731,57485ef0dfb7b83ecce1f01c,57bd40eb1a68b1000b1ac21c,0,55.220470,0,1.1. Properties of random samples
5,579632509288d4000b36ae66,1472020876693,1472020876700,57485ef0dfb7b83ecce1f01c,57bd41811a68b1000b1aed86,0,10.999542,1,1.1. Properties of random samples
6,579632509288d4000b36ae66,1472021076795,1472021076808,57485ef0dfb7b83ecce1f01c,57bd41811a68b1000b1aed86,1,168.382965,1,1.1. Properties of random samples
7,579632509288d4000b36ae66,1472021148468,1472021148475,57485ef0dfb7b83ecce1f01c,57bd41811a68b1000b1aed86,2,56.591736,1,1.1. Properties of random samples
8,579632509288d4000b36ae66,1472021184524,1472021184539,57485ef0dfb7b83ecce1f01c,57bd41811a68b1000b1aed86,3,25.226973,1,1.1. Properties of random samples
9,579632509288d4000b36ae66,1472021245199,1472021245208,57485ef0dfb7b83ecce1f01c,57bd41811a68b1000b1aed86,4,9.726890,1,1.1. Properties of random samples


In [9]:
12714*9

114426

In [30]:
list_capsules = dict(properties['capsule name'].value_counts()).keys()

In [86]:
properties.groupby(["test attempt"]).first()

,capsule name,map name,user id,question number,success,user id
test attempt,,,,,,
57bace537fc49a000bd84221,Random experiment and sample space,1.1. Properties of random samples,564f68e80a728f08a779fe6e,0,0,564f68e80a728f08a779fe6e
57bc5d981a68b1000b171473,Random experiment and sample space,1.1. Properties of random samples,564f68e80a728f08a779fe6e,0,0,564f68e80a728f08a779fe6e
57bd40eb1a68b1000b1ac21c,Random experiment and sample space,1.1. Properties of random samples,57485ef0dfb7b83ecce1f01c,0,0,57485ef0dfb7b83ecce1f01c
57bd41811a68b1000b1aed86,Random experiment and sample space,1.1. Properties of random samples,57485ef0dfb7b83ecce1f01c,0,1,57485ef0dfb7b83ecce1f01c
57bd44041a68b1000b1b0c38,Random experiment and sample space,1.1. Properties of random samples,57485ef0dfb7b83ecce1f01c,0,0,57485ef0dfb7b83ecce1f01c
57bd44101a68b1000b1b0cc2,Random experiment and sample space,1.1. Properties of random samples,57485ef0dfb7b83ecce1f01c,0,1,57485ef0dfb7b83ecce1f01c
57bd44131a68b1000b1b0d00,Random experiment and sample space,1.1. Properties of random samples,5762d0afdfb7b870f9ae71de,0,1,5762d0afdfb7b870f9ae71de
57bd44f31a68b1000b1b2b86,Random Variables,1.1. Properties of random samples,5762d0afdfb7b870f9ae71de,0,1,5762d0afdfb7b870f9ae71de
57bd450d1a68b1000b1b2e9c,Random experiment and sample space,1.1. Properties of random samples,57485ef0dfb7b83ecce1f01c,0,1,57485ef0dfb7b83ecce1f01c


In [138]:
properties=properties[["capsule name", 'test attempt', "map name", "question number", "success", "user id"]]
list_answers_test = {}
forbidden_test_attempt = []
for capsule in list_capsules:
    right_capsule = properties["capsule name"] == capsule
    number_question = max(properties[right_capsule]["question number"])+1
    every_q = (properties["question number"]) == number_question-1 
    #test_attempts = set(properties[right_capsule][every_q]["test attempt"])
    test_attempts = []
    for user in set(properties[right_capsule]["user id"]):
        try:
            test_attempts.append(properties[right_capsule][every_q][properties["user id"] == user]["test attempt"].reset_index(drop=True)[0])
        except:
            test_attempts += []
    forbidden_test_attempt += test_attempts
    number_test_attempts = len(test_attempts)
    list_answers_test[capsule] = np.zeros((number_test_attempts, number_question))
    for (index, test_attempt) in enumerate(test_attempts):
        right_test_attempt = properties["test attempt"] == test_attempt
        for i in range(number_question):
            right_question = properties["question number"] == i
            list_answers_test[capsule][index][i] = list(properties[right_capsule][right_test_attempt][right_question]["success"])[0]


/home/enguerand/.virtualenvs/gryzz-lab/lib/python2.7/site-packages/ipykernel/__main__.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/enguerand/.virtualenvs/gryzz-lab/lib/python2.7/site-packages/ipykernel/__main__.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [157]:
set([1,2])-set([3])

{1, 2}

In [162]:

list_answers = {}
for capsule in list_capsules:
    right_capsule = properties["capsule name"] == capsule
    number_question = max(properties[right_capsule]["question number"])+1
    every_q = (properties["question number"]) == number_question-1 
    test_attempts = set(properties[right_capsule][every_q]["test attempt"])
    number_test_attempts = len(test_attempts)
    list_answers[capsule] = np.zeros((number_test_attempts, number_question))
    for (index, test_attempt) in enumerate(test_attempts):
        right_test_attempt = properties["test attempt"] == test_attempt
        for i in range(number_question):
            right_question = properties["question number"] == i
            list_answers[capsule][index][i] = list(properties[right_capsule][right_test_attempt][right_question]["success"])[0]


/home/enguerand/.virtualenvs/gryzz-lab/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/enguerand/.virtualenvs/gryzz-lab/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [163]:
number_users = min([len(a) for a in list_answers.values()])
list_answers_final = list_answers

In [165]:
list_answers_final_num = {}
list_answers_final_test = {}

list_answers_final_num[0] = list_answers["Random experiment and sample space"]
list_answers_final_num[1] = list_answers["Random Variables"]
list_answers_final_num[2] = list_answers["Multiple random variables and independence"]
list_answers_final_num[3] = list_answers["Expectation and other Moments"]
list_answers_final_num[4] = list_answers["Conditional distributions"]
list_answers_final_num[5] = list_answers["Conditional Moments"]
list_answers_final_num[6] = list_answers["Random sample"]

list_answers_final_test[0] = list_answers_test["Random experiment and sample space"][:10]
list_answers_final_test[1] = list_answers_test["Random Variables"][:10]
list_answers_final_test[2] = list_answers_test["Multiple random variables and independence"][:10]
list_answers_final_test[3] = list_answers_test["Expectation and other Moments"][:10]
list_answers_final_test[4] = list_answers_test["Conditional distributions"][:10]
list_answers_final_test[5] = list_answers_test["Conditional Moments"][:10]
list_answers_final_test[6] = list_answers_test["Random sample"][:10]

In [40]:
list_answers_final_num_train = {}
list_answers_final_num_test = {}
for i in range(7):
    list_answers_final_num_train[i] = list_answers_final_num[i][:-10]
    list_answers_final_num_test[i] = list_answers_final_num[i][-10:]

In [166]:
import pickle
output = open('/home/enguerand/Documents/ENSAE/2A /Césure/Gryzz-Lab/Data Analysis/Placement test/IRT/list_answers_final_num_train.pkl', 'wb')

pickle.dump(list_answers_final_num, output)
output.close()

output = open('/home/enguerand/Documents/ENSAE/2A /Césure/Gryzz-Lab/Data Analysis/Placement test/IRT/list_answers_final_num_test.pkl', 'wb')
pickle.dump(list_answers_final_test, output)
output.close()


In [108]:
pkl_file = open('list_answers_final_num.pkl', 'rb')

data1 = pickle.load(pkl_file)
(data1)

{0: array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.],
        [ 0.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.],
        [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  1.,  1.,  0.

In [107]:
pkl_file.close()